## Setup

In [1]:
# !pip install pyarrow
# !pip install googletrans==4.0.0-rc1
# !pip install tqdm
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !pip install openpyxl

import math
import pandas as pd
from googletrans import Translator
from tqdm import tqdm as tqdm

In [2]:
class Preprocessor:
    
    def __init__(self, df):
        i = 0
        while i < df.shape[0]:
            if isinstance(df.iloc[i, 0], str) and df.iloc[i, 0][:23] == "Օգտագործվող հապավումներ":
                break
            i += 1
        i += 1
        abbrev_list_begin_i = i
        while i < df.shape[0]:
            if df.iloc[i].isna().iloc[0]:
                break
            i += 1
        abbrev_list_end_i = i

        abbrevs = df.iloc[abbrev_list_begin_i: abbrev_list_end_i, :2]
        # make a dict of abbreviations and their openings
        self.abbrev_dict = dict(zip(abbrevs.iloc[:, 0], abbrevs.iloc[:, 1]))

        while i < df.shape[0]:
            if isinstance(df.iloc[i, 0], str) and df.iloc[i, 0][:6] == "Օբյեկտ":
                break
            i += 1

        self.df_hy = df.rename(columns=dict(zip(df.columns, df.iloc[i]))).iloc[i+1:, :3]
        self.translator = Translator()

    
    def replace_an_abbrev_in_a_cell(self, text, abbrev):
        ''' text: str
            abbrev: str
            replaces abbrev's incidences in text according to abbrev_dict'''
    
        if isinstance(text, float) and math.isnan(text):
            return text

        text_split = text.split(' ')
        # to count for cases such as 'ՏԳՏԴ խմբ. 2'
        abbrev_split = abbrev.split(' ')
        
        for i in range(len(text_split)):
            
            abbrev_start_index = i
            abbrev_suffix = None
            for abbrev_part in abbrev_split:
                # to count for cases such as 'ՏԳՏՀԴ-ի'
                word_parts = text_split[i].split('-')
                if abbrev_part != word_parts[0]:
                    break
                if len(word_parts) > 1 and abbrev_suffix is None:
                    abbrev_suffix = word_parts[1]
                i += 1
                
            # in case of coincidence replace the abbreviation
            if len(abbrev_split) == i - abbrev_start_index:
                text_split[abbrev_start_index] = self.abbrev_dict[abbrev]
                if abbrev_suffix:
                    text_split[abbrev_start_index] += '-' + abbrev_suffix
                for j in range(abbrev_start_index + 1, i):
                    text_split[j] = ''
    
        return ' '.join(text_split)

    
    def replace_all_abbrevs_in_a_cell(self, text):
        for abbrev in self.abbrev_dict.keys():
            text = self.replace_an_abbrev_in_a_cell(text, abbrev)
        return text


    @classmethod
    def apply_op_optimized_to_col(cls, col, op, col_new=None, verbose=False):
        ''' col: pd.Series
            op: function
            col_new: pd.Series
            verbose: bool
            Applies op to each element of col so that for repeated
            contiguous elements of col op is not calculated twice.
            The result is written in col_new. '''
        if col_new is None:
            col_new = col
            
        last_elem = None
        last_elem_modified = None
        col_len = len(col)
        i = 0
        while i < col_len:
            while i < col_len and last_elem == col.iloc[i]:
                col_new.iloc[i] = last_elem_modified
                i += 1
                if verbose and i % 50 == 0:
                    print('row', i)
            if i < col_len:
                last_elem = col.iloc[i]
                col_new.iloc[i] = op(col.iloc[i])
                last_elem_modified = col_new.iloc[i]
            i += 1
            if verbose and i % 50 == 0:
                print('row', i)


    def apply_op_optimized(self, df, op, df_new=None, verbose=False):
        ''' df: pd.DataFrame
            op: function
            df_new: pd.DataFrame
            Applies apply_op_optimized_to_col to each column of df.
            The result is written in df_new. '''
        if df_new is None:
            df_new = df
        for i in range(df.shape[1]):
            Preprocessor.apply_op_optimized_to_col(df.iloc[:, i], op, col_new=df_new.iloc[:, i], verbose=verbose)

    
    def replace_all_abbrevs(self):
        self.apply_op_optimized(self.df_hy, self.replace_all_abbrevs_in_a_cell)

    
    def translate_a_cell(self, text, src, dest):
        while True:
            # to suppress an unknown exception error in googletrans
            try:
                return self.translator.translate(text, src=src, dest=dest).text
            except Exception as e:
                print(f"Error occurred: {e}")
                print("Retrying...")

    
    def translate_whole(self, df, src, dest, verbose=True):
        def translate_src_dest(text):
            return self.translate_a_cell(text, src=src, dest=dest)
        df_translated = pd.DataFrame(index=df.index, columns=df.columns)
        self.apply_op_optimized(df, translate_src_dest, df_new=df_translated, verbose=verbose)
        return df_translated

In [3]:
df = pd.read_excel("C:/Users/saro.harutyunyan/Downloads/DataMapping_TOTAL.xlsx", sheet_name="Տրամադրում")

## Preprocessing

In [4]:
preprocessor = Preprocessor(df)

In [5]:
preprocessor.df_hy.head()

,Օբյեկտ \nObject,Հատկանիշ\nAttribute,Մեկնաբանություն\nComment
59,ՓՄՁ,Տնտեսավարողների քանակն ըստ տնտեսական գործունեո...,NaN
60,ՓՄՁ,Տնտեսավարողների քանակն ըստ ՀՀ մարզերի և Երևան ...,NaN
61,ՓՄՁ,Տնտեսավարողների կողմից արտադրված ապրանքների ու...,NaN
62,ՓՄՁ,Տնտեսավարողների կողմից արտադրված ապրանքների ու...,NaN
63,ՓՄՁ,Վարձու աշխատողների թվաքանակն ըստ տնտեսական գոր...,NaN


In [6]:
preprocessor.abbrev_dict

{'ԲՊՌ': 'Բնակչության պետական ռեգիստր',
 'ՌԵՍՊՈՆԴԵՆՏ': 'Վիճակագրական դիտարկման/նպատակային հետազոտության իրականացման նպատակով Արմստատի կողմից իրազեկված իրավաբանական անձինք և անհատ ձեռնարկատերերը, իրավաբանական անձանց առանձնացված ստորաբաժանումները, Հայաստանի Հանրապետության տարածքում գործող օտարերկրյա իրավաբանական անձինք, ինչպես նաև դրանց՝ Հայաստանի Հանրապետության տարածքում գործող առանձնացված ստորաբաժանումները, պետական կառավարման համակարգի մարմինները, տեղական ինքնակառավարման մարմինները, համայնքային կառավարչական հիմնարկները, Հայաստանի Հանրապետություն ժամանած կամ Հայաստանի Հանրապետությունում գտնվող ֆիզիկական անձինք` անկախ քաղաքացիությունից և քաղաքացիություն ունենալու հանգամանքից, կամ տնային տնտեսությունները, որոնց ընդգրկման շրջանակը կախված դիտարման/ նպատակային հետազոթյան օբյեկտից կարող է տարբեր լինել:',
 'ՏԳՏԴ խմբ. 2': 'Տնտեսական գործունեության տեսակների դասակարգչի 2-րդ խմբագրություն',
 'ՊԵԿ': 'Պետական եկամուտների կոմիտե',
 'Տնտեսավարողներ': 'ՌԵՍՊՈՆԴԵՆՏՆԵՐ՝ ամբողջականացված ՊԵԿՀՎ-ների վերաբերյ

In [7]:
# modify the dict according to needs
preprocessor.abbrev_dict['ՓՄՁ'] = 'Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ'

keys_to_delete = ['ՌԵՍՊՈՆԴԵՆՏ',
                  'Տնտեսավարողներ',
                  'ՏԳՏԴ խմբ. 2-ի G (ՋԻ) բաժին',
                  'ՏԳՏԴ խմբ. 2-ի G (ՋԻ) բաժնի 45 ծածկագիր',
                  'ՏԳՏԴ խմբ. 2-ի G (ՋԻ) բաժնի 46 ծածկագիր',
                  'ՏԳՏԴ խմբ. 2-ի G (ՋԻ) բաժնի 47 ծածկագիր']

for key in keys_to_delete:
    del preprocessor.abbrev_dict[key]

In [8]:
preprocessor.abbrev_dict

{'ԲՊՌ': 'Բնակչության պետական ռեգիստր',
 'ՏԳՏԴ խմբ. 2': 'Տնտեսական գործունեության տեսակների դասակարգչի 2-րդ խմբագրություն',
 'ՊԵԿ': 'Պետական եկամուտների կոմիտե',
 'ՊԵԿՀՎ': 'ՊԵԿ-ում հաշվառված հարկային հաշվարկ ներկայացնող սուբյեկտներ',
 'Արմստատ': 'Վիճակագրական կոմիտե',
 'ՓՄՁ': 'Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ',
 'ԻԱՊՌՍ': 'ԻԱՊՌ-ում գրացված/հաշվառված իրավաբանական անձ, Իրավաբանական անձի առանձնացված ստորաբաժանում, հիմնարկ, այդ թվում՝ Պետական կառավարման համակարգի մարմիններ, տեղական ինքնակառավարման մարմիններ, համայնքային կառավարչական հիմնարկներ և այլն',
 'ԻԱՊՌ': 'Իրավաբանական անձանց պետական ռեգիստր',
 'ԿԲ': 'Կենտրոնական բանկ',
 'ՀՏՕ': 'Հյուրանոցային տնտեսության օբյեկտ',
 'ԶՕ': 'Զբոսաշրջային օբյեկտ',
 'ԶԳ': 'Զբոսաշրջային գործակալություն',
 'ԾՄԱԸԽ': 'Ծառայությունների միջազգային առևտրի ընդլայնված խմբավորում',
 'ԱՏԳԱԱ': 'Արտաքին տնտեսական գործունեության ապրանքային ծածկագիր',
 'ԵԱՏՄ ': 'Եվրասիական տնտեսական միություն',
 'ՀՓԻ': 'Հատուկ փոխառության իրավունք',
 'ՃՏՊ': 'Ճանապարհատրանսպորտայ

In [9]:
preprocessor.replace_all_abbrevs_in_a_cell('ՀՀ առևտրի շրջանառությունը, ընդամենը՝ ըստ ՏԳՏԴ խմբ. 2-ի G (ՋԻ) բաժնի 45, 46, 47 ծածկագրերի ')

'Հայաստանի Հանրապետություն առևտրի շրջանառությունը, ընդամենը՝ ըստ Տնտեսական գործունեության տեսակների դասակարգչի 2-րդ խմբագրություն-ի   G (ՋԻ) բաժնի 45, 46, 47 ծածկագրերի '

In [10]:
preprocessor.replace_all_abbrevs()

In [11]:
preprocessor.df_hy.head(49)

,Օբյեկտ \nObject,Հատկանիշ\nAttribute,Մեկնաբանություն\nComment
59,Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ,Տնտեսավարողների քանակն ըստ տնտեսական գործունեո...,NaN
60,Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ,Տնտեսավարողների քանակն ըստ Հայաստանի Հանրապետո...,NaN
61,Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ,Տնտեսավարողների կողմից արտադրված ապրանքների ու...,NaN
62,Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ,Տնտեսավարողների կողմից արտադրված ապրանքների ու...,NaN
63,Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ,Վարձու աշխատողների թվաքանակն ըստ տնտեսական գոր...,NaN
64,Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ,Վարձու աշխատողների թվաքանակն ըստ Հայաստանի Հան...,NaN
65,Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ,Միջին ամսական անվանական աշխատավարձն ըստ տնտես...,NaN
66,Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ,Միջին ամսական անվանական աշխատավարձն ըստ Հայա...,NaN
67,Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ,Համախառն ավելացված արժեքն ըստ տնտեսական գործու...,"մլն. դրամ, հիմնական գներով, %-ով՝ ընդամենի նկա..."
68,Փոքր ու միջին ձեռնարկատիրության սուբյեկտներ,Համախառն ավելացված արժեքն ըստ Տնտեսական գործու...,"մլն. դրամ, հիմնական գներով, %-ով՝ գործունեությ..."


## Translation

In [12]:
# this may take a while to be completed
df_en = preprocessor.translate_whole(preprocessor.df_hy, 'hy', 'en')

row 50
row 100
row 150
row 200
row 250
row 300
row 350
row 400
row 450
row 500
row 550
row 600
row 650
row 700
row 750
row 800
row 850
row 900
row 950
row 1000
row 1050
row 1100
row 1150
row 1200
row 1250
row 1300
row 1350
row 1400
row 1450
row 1500
row 1550
row 1600
row 1650
row 1700
row 1750
row 1800
row 1850
row 1900
row 1950
row 2000
row 50
row 100
Error occurred: 'Translator' object has no attribute 'raise_Exception'
Retrying...
Error occurred: 'Translator' object has no attribute 'raise_Exception'
Retrying...
Error occurred: 'Translator' object has no attribute 'raise_Exception'
Retrying...
Error occurred: 'Translator' object has no attribute 'raise_Exception'
Retrying...
Error occurred: 'Translator' object has no attribute 'raise_Exception'
Retrying...
Error occurred: 'Translator' object has no attribute 'raise_Exception'
Retrying...
Error occurred: 'Translator' object has no attribute 'raise_Exception'
Retrying...
Error occurred: 'Translator' object has no attribute 'raise_Exce

In [13]:
with pd.ExcelWriter("C:/Users/saro.harutyunyan/Downloads/DataMapping_TOTAL.xlsx", engine='openpyxl', mode='a') as writer:
    df_en.to_excel(writer, sheet_name='Provision', index=False)

In [15]:
# dict(zip(df_hy.iloc[:, 0], df_en.iloc[:, 0]))